# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [41]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable 

#### Creating list of filepaths to process original event csv data files

In [42]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [43]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
#         print(row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16])
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [44]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [45]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print("---- Successfully accessed to cassandra cluster ----")
except Exception as e:
    print(e)

---- Successfully accessed to cassandra cluster ----


#### Create Keyspace

In [46]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION =
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
    
except Exception as e:
    print(e)

#### Set Keyspace

In [47]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [62]:
## Query 1: Give me the artist, song title and song's length in the music app history that was heard during
## sessionId = 338, and itemInSession = 4
## For table artist_song_length, sessionId was used as a partition key because the queries will filter by this column. itemInSession was used as clustering columns to help make up a unique key.

In [63]:
query = "CREATE TABLE IF NOT EXISTS artist_song_length "
query = query + "(sessionId int, itemInSession int, artist text, song text, length double, \
                PRIMARY KEY (sessionId, itemInSession))"
try:
    result = session.execute(query)
    print(result)
except Exception as e:
    print(e)
              

In [64]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_length (sessionId,  itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),  int(line[3]), line[0],  line[9], float(line[5])))                        

#### Do a SELECT to verify that the data have been inserted into each table

In [65]:
query = "SELECT artist, song, length \
        FROM artist_song_length \
        WHERE sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
    
    records = PrettyTable()
    records.field_names = ["artist", "song", "length"]
    for row in rows:
        records.add_row([row.artist, row.song, row.length])
        
    print(records)
except Exception as e:
    print(e)

+-----------+---------------------------------+----------+
|   artist  |               song              |  length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [66]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
## For table artist_song_user, userId and sessionId were used as a partition key because the queries will filter by this column. itemInSession was used as clustering columns to sort by it.

In [67]:
query = "CREATE TABLE IF NOT EXISTS artist_song_user "
query = query + "(userId int, sessionId int, itemInSession int, artist text, firstName text, lastName text, song text, \
                PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    result = session.execute(query)
    print(result)
except Exception as e:
    print(e)

In [68]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, firstName, lastName, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))  

In [69]:
query = "SELECT artist, song, firstName, lastName, itemInSession \
        FROM artist_song_user \
        WHERE userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
    
    records = PrettyTable()
    records.field_names = ["artist", "song", "firstName", "lastName", "itemInSession"]
    for row in rows:
#         print(row)
        records.add_row([row.artist, row.song, row.firstname, row.lastname, row.iteminsession])
        
    print(records)
except Exception as e:
    print(e)    

+-------------------+------------------------------------------------------+-----------+----------+---------------+
|       artist      |                         song                         | firstName | lastName | itemInSession |
+-------------------+------------------------------------------------------+-----------+----------+---------------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |       0       |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |       1       |
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |       2       |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |       3       |
+-------------------+------------------------------------------------------+-----------+----------+---------------+


In [70]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## For table users, song was used as a partition key because the queries will filter by this column. userId was used as clustering columns to help make up a unique key.

In [71]:
query = "CREATE TABLE IF NOT EXISTS users "
query = query + "(song text, userId int, firstName text, lastName text, \
                PRIMARY KEY (song, userId))"
try:
    result = session.execute(query)
    print(result)
except Exception as e:
    print(e)         

In [72]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))  

In [73]:
query = "SELECT firstName, lastName \
        FROM users \
        WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
    
    records = PrettyTable()
    records.field_names = ["firstName", "lastName"]
    for row in rows:
#         print(row)
        records.add_row([row.firstname, row.lastname])
        
    print(records)
except Exception as e:
    print(e)  

+------------+----------+
| firstName  | lastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [74]:
query = "DROP TABLE artist_song_length"
query2 = "DROP TABLE artist_song_user"
query3 = "DROP TABLE users"
try:
    print("drop artist_song_length ", session.execute(query))
    print("drop artist_song_user ", session.execute(query2))
    print("drop users ", session.execute(query3))
except Exception as e:
    print(e)


drop artist_song_length  <cassandra.cluster.ResultSet object at 0x7f553ec26cc0>
drop artist_song_user  <cassandra.cluster.ResultSet object at 0x7f553ec2eda0>
drop users  <cassandra.cluster.ResultSet object at 0x7f55385754e0>


### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()